# Coursera Capstone Project - IBM Data Science Specialization
## Project Title

Analysis of St. Louis City neighborhoods based on data related to demographic, crime and places of interest.

### Import packages

In [340]:
#import the packages
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import requests
import folium
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import os
import math
from os import listdir
from os.path import isfile, join

### Define Constants

In [341]:
#define constants
BASE_URL = 'https://api.foursquare.com/v2'
CLIENT_ID = ''
CLIENT_SECRET = ''
wikiUrl = 'https://en.wikipedia.org/wiki/List_of_neighborhoods_of_St._Louis'
V='20180604'

## 1. Intial DataFrome of Neighborhood with long/lat and demographic data
### Build Initial DataFrame with WikiPedia Table Data

In [439]:
#Fetch Wikipedia article
resp = requests.get(wikiUrl)
html_text = resp.text

In [440]:
#Parse and build data frame
soup = BeautifulSoup(html_text, 'html.parser')
neighborhood_table = soup.find_all('table')[1]
rows = neighborhood_table.find_all('tr')
header = []
for th in rows[0].find_all('th'):
    header.append(th.text.replace('\n',''))
df_neighborhood = pd.DataFrame(columns = header)
rows = rows[1:]
rownum  = 0
for row in rows:
    df_row = []
    for td in row.find_all('td'):
        df_row.append(td.text.replace('\n',''))
    #print(df_row)
    df_neighborhood.loc[rownum] = df_row
    rownum = rownum + 1
df_neighborhood.shape

(79, 9)

### Extract Longitude and latitude and append to the dataframe

In [441]:
geolocator = Nominatim(user_agent='stl_data_analysis', format_string="%s, St. Louis Missouri")
df_coords = pd.DataFrame(columns = ['Neighborhood','Latitude', 'Longitude'])
#df_neighborhood.set_index('Neighborhood', inplace = True)
#df_neighborhood.head()
row_num = 0
for entry in df_neighborhood['Neighborhood']:
     #print('Getting entry for {}'.format(entry))
     data = geolocator.geocode(entry)
     if data != None:
         address, (latitude, longitude) = data
         df_coords.loc[row_num] = [entry,latitude,longitude]
         #print(entry,address, latitude, longitude)
     else:
         print('Not found for entry - {}'.format(entry))
         #df_coords.loc[row_num] = [entry,99.99,99.99]
     row_num =row_num+1
df_coords.shape

Not found for entry - Clayton-Tamm
Not found for entry - College Hill
Not found for entry - Downtown West
Not found for entry - The Gate District
Not found for entry - Grand Center
Not found for entry - Hamilton Heights
Not found for entry - JeffVanderLou
Not found for entry - Kings Oak
Not found for entry - Lafayette Square
Not found for entry - LaSalle Park
Not found for entry - Marine Villa
Not found for entry - Mark Twain
Not found for entry - Mark Twain/I-70 Industrial
Not found for entry - McKinley Heights
Not found for entry - Mount Pleasant
Not found for entry - North Hampton
Not found for entry - Old North St. Louis
Not found for entry - Peabody Darst Webbe
Not found for entry - Princeton Heights
Not found for entry - South Hampton
Not found for entry - Southwest Garden
Not found for entry - Wydown/Skinker


(57, 3)

In [442]:
df_merged = df_neighborhood
df_merged = df_merged.join(df_coords.set_index('Neighborhood'), on='Neighborhood')
df_merged.shape

(79, 11)

### Write to CSV so that manual update for missing coordinates can be done

In [443]:
df_merged.to_csv('df_merged.csv')

### Manual Data cleaning
For Below Neighbourhoods geopy geolocator could not find the coordinates. For these manual google search was done. Even 2 out of these (`The Gate District`,`Peabody Darst Webbe`) dint have results in google. For these two random coordinates were taken from related geojson file.

* List of neighbourhoods missing coordinates.
Not found for entry - Clayton-Tamm
Not found for entry - College Hill
Not found for entry - Downtown West
Not found for entry - The Gate District
Not found for entry - Grand Center
Not found for entry - Hamilton Heights
Not found for entry - JeffVanderLou
Not found for entry - Kings Oak
Not found for entry - Lafayette Square
Not found for entry - LaSalle Park
Not found for entry - Marine Villa
Not found for entry - Mark Twain
Not found for entry - Mark Twain/I-70 Industrial
Not found for entry - McKinley Heights
Not found for entry - Mount Pleasant
Not found for entry - North Hampton
Not found for entry - Old North St. Louis
Not found for entry - Peabody Darst Webbe
Not found for entry - Princeton Heights
Not found for entry - South Hampton
Not found for entry - Southwest Garden
Not found for entry - Vandenventer
Not found for entry - Wydown/Skinker

### Read Final Updated CSV to go further

In [449]:
df_merged = pd.read_csv('df_merged_final.csv')

In [451]:
df_merged.drop(df_merged.columns[0], axis=1, inplace = True)
df_merged.head()

,Neighborhood,Population,White,Black,Hispanic/Latino2,AIAN1,Asian,Mixed Race,Corridor,Latitude,Longitude
0,Academy,"3,006",16.9,54.7,20.5,1.52,4.3,3.5,North,38.659025,-90.267293
1,Baden,"7,268",6.3,91.8,0.5,0.10,0.0,1.3,North,38.709732,-90.230831
2,Benton Park,"3,532",68.2,25.1,3.2,0.30,1.2,3.8,South,38.597383,-90.222621
3,Benton Park West,"4,404",28.0,59.6,10.5,0.00,1.9,5.1,South,38.597383,-90.222621
4,Bevo Mill,"12,654",74.2,13.8,7.5,0.40,4.6,3.9,South,38.581723,-90.266955


### Plot the neighbourhoods on map using geojson

In [452]:
# Get log/lat of St. Louis
address_stl = 'St. Louis, US'
geolocator_stl = Nominatim(user_agent="stl_explorer")
location = geolocator_stl.geocode(address_stl)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 38.6268039, -90.1994097.


In [453]:
map_stl = folium.Map(location=[latitude, longitude], zoom_start=12)
geo_json = os.path.join('.','st-louis.geojson')
# add markers to map
    
folium.GeoJson(
    geo_json,
    name='geojson'
).add_to(map_stl)    

for lat, lng, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_stl)

map_stl

## 2. St. Louis Crime Data Extraction
#### St. Louis Police Department ID to Neighborhood mapping
St. Louis Crime Report data (available at this [link](https://www.slmpd.org/Crimereports.shtml) has `Neighborhood` as numeric value. They have a reference sheet in [FAQ](http://www.slmpd.org/Crime/CrimeDataFrequentlyAskedQuestions.pdf) document which lists ID to Neighborhood mapping. The next step in this project is to load the mapping from this document, join with `df_merged` and clean any data which is missing/different.

In [454]:
#Load the ID to Neighborhood mapping
df_id_mapping = pd.read_csv('Id_to_Neighbourhood_Mapping.csv')

# join with df_merged on neighborhood column
df_new = df_merged.join(df_id_mapping.set_index('Neighborhood'), on='Neighborhood')

#Rows where data is still missing
df_new[df_new['Neighborhood_ID'].isnull()]
missing_neighborhood_ids= {
    'Botanical Heights':28,
    'Carondelet': 1,
    'Fairground':83,
    'Forest Park Southeast':39,
    'Grand Center':77,
    'Greater Ville':56,
    'Hi-Pointe':44,
    'JeffVanderLou':59,
    'LaSalle Park':34,
    'Mark Twain/I-70 Industrial':70,
    'Near North Riverfront':79,
    'O\'Fallon':85,
    'Wells/Goodfellow':50,
    'Wydown/Skinker':45
}

len(missing_neighborhood_ids)
for key,value in missing_neighborhood_ids.items():
    #print('{},{}'.format(key,value))
    df_new.loc[df_new.Neighborhood == key,'Neighborhood_ID' ] = value
df_merged = df_new
df_merged.shape

(79, 12)

#### St. Louis City Crime data for the year 2018
For the year of 2018, crime data was obtined from St. Louis Metro Police Dept [website](https://www.slmpd.org/Crimereports.shtml). In total there were 46,742 crimes of various categories. We will get into details of it as we go.
Monthly crime records were download and are avaiable in Github in crime_records folder.
These will be verified for shape and meragebility into a single dataset.

In [455]:
#initialize merged_crime_df with columns
JanCSV = pd.read_csv('./crime_records/January2018.CSV')
# read the dir crime_records for CSVs and merge into a single dataframe.
merged_crime_df = pd.DataFrame(columns=JanCSV.columns.values)
mypath = './crime_records/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for f in onlyfiles:
    file_df = pd.read_csv('{}{}'.format(mypath,f))
    merged_crime_df = merged_crime_df.append(file_df)
    #print(merged_crime_df.shape)
merged_crime_df.shape

(46742, 20)

### So far we have following data frames
* `df_merged` - Neighborhood data with long/lat, demographics and NeighborhoodIds
* `merged_crime_df` - 2018 Crime Data, not grouped by neighbourhood yet.
* The next step will be collect neighborhood data from foursquare API.

But before that we should perform exploratory data analysis of crime vs neighborhood for St. Louis and show the results in a choropleth map.


In [458]:
#df_merged = df_merged.astype({'Neighborhood_ID':'int64'})
#merged_crime_df = merged_crime_df.astype({'Neighborhood':'int64'})

In [459]:
def updateNeighborHood(row,df_merged=df_merged):
    for idx, d in df_merged.iterrows():
        if d.Neighborhood_ID == row.Neighborhood:
            row.Neighborhood = d.Neighborhood
            #print('updated', row.Neighborhood)
            return row
    return row

merged_crime_df = merged_crime_df.apply(updateNeghiborHood, axis = 1)




In [422]:
df_crime_stl_grped = merged_crime_df.groupby('Neighborhood').count().sort_values(by='Count',ascending=False)

In [423]:
# Plot the Crime count of Choropleth map
# Plot White vs Black in Choropleth map
# Correct coorindates for outlier neighborhoods
df_crime_stl_grped.reset_index(inplace=True)
df_crime_stl_grped.head()

,Neighborhood,Complaint,CodedMonth,DateOccur,FlagCrime,FlagUnfounded,FlagAdministrative,Count,FlagCleanup,Crime,District,Description,ILEADSAddress,ILEADSStreet,LocationName,LocationComment,CADAddress,CADStreet,XCoord,YCoord
0,Downtown,3457,3457,3457,3457,3457,3457,3457,3457,3457,3457,3457,3457,3457,3450,3457,3457,3457,3457,3457
1,Dutchtown,2402,2402,2402,2402,2402,2402,2402,2402,2402,2402,2402,2402,2402,2401,2402,2402,2402,2402,2402
2,Central West End,2062,2062,2062,2062,2062,2062,2062,2062,2062,2062,2062,2062,2062,2062,2062,2062,2062,2062,2062
3,Downtown West,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866
4,Carondelet,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605,1605


In [429]:
df_crime_stl_grped[df_crime_stl_grped['Neighborhood'] == 'O’Fallon']

,Neighborhood,Complaint,CodedMonth,DateOccur,FlagCrime,FlagUnfounded,FlagAdministrative,Count,FlagCleanup,Crime,District,Description,ILEADSAddress,ILEADSStreet,LocationName,LocationComment,CADAddress,CADStreet,XCoord,YCoord
82,O’Fallon,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49


In [425]:
map_stl = folium.Map(location=[latitude, longitude], zoom_start=12)
geo_json = os.path.join('.','st-louis.geojson')
# add markers to map
    
# folium.GeoJson(
#     geo_json,
#     name='geojson'
# ).add_to(map_stl)    

map_stl.choropleth(
    geo_data=geo_json,
    data=df_crime_stl_grped,
    columns=['Neighborhood', 'Count'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='St. louis crimee stats'
)

for lat, lng, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_stl)

map_stl

## 3. Venues data for location from foursquare

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']